In [149]:
import math
from scipy.stats import rankdata
import pandas as pd

In [263]:
Totals = [4, 15, 33, 61, 95] # Threshold for new row. For instance, more than 15 seats means 3 rows

sumdelegates = 24
rows = 3

# Maximum radius of spot is 0.5/rows; leave a bit of space.
radius = 0.4/rows

# Create list of centre spots
poslist_24 = []
poslist_30 = []
poslist_40 = []
poslist_44 = []
poslist_50 = []
poslist_60 = []
poslist_70 = []
poslist_80 = []

for i in range(1, rows):
    """ The relevant math is borrowed from David Richfield (slashme on GitHub)"""
        
    # Each row can contain pi/(2asin(2/(3n+4i-2))) spots, where n is the number of rows and i is the number
    # of the current row. Fill each row proportionally to the "fullness" of the diagram, up to the second-last row.
        
    J = int(float(sumdelegates) / Totals[rows-1] * math.pi/(2*math.asin(2.0/(3.0*rows+4.0*i-2.0))))
        
    # The radius of the ith row in an N-row diagram (Ri) is (3*N+4*i-2)/(4*N)
        
    R = (3.0*rows+4.0*i-2.0)/(4.0*rows)
        
    if J == 1:
        poslist.append([math.pi/2.0, 1.75*R, R])
    else:
        for j in range(J):
            # The angle to a spot is n.(pi-2sin(r/Ri))/(Ni-1)+sin(r/Ri) where Ni is the number in the arc
            # x=R.cos(theta) + 1.75
            # y=R.sin(theta)
                
            angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J)-1.0)+math.sin(radius/R)   
            poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])

                
# Now whatever seats are left go into the outside row:
J_2 = sumdelegates-len(poslist)
R = (7.0*rows-2.0)/(4.0*rows)
if J_2 == 1:
    poslist.append([math.pi/2.0, 1.75*R, R])
else:
    for j in range(J_2):
        angle = float(j) * (math.pi-2.0*math.sin(radius/R)) / (float(J_2)-1.0)+math.sin(radius/R)
        poslist_24.append([angle, R*math.cos(angle)+1.75, R*math.sin(angle)])
poslist_24.sort(reverse = True)

# Pure seats allocation -> add the party that holds the seat
dataframe = pd.DataFrame(data=poslist_24, columns = ['Angle', 'x', 'y'])
party = pd.Series(['A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C',
                   'D', 'D', 'E', 'F'])
df = pd.concat([dataframe, party], axis = 1, ignore_index=True)
df.columns = ['Rank', 'x', 'y', 'party']
df

,Rank,x,y,party
0,3.057482,0.172264,0.133019,A
1,3.035128,0.507077,0.132829,A
2,2.996650,0.842945,0.132399,A
3,2.727107,0.300738,0.637639,A
4,2.616748,0.668248,0.626349,A
5,2.426309,1.058002,0.601179,A
6,2.396733,0.585961,1.073292,A
7,2.198367,1.016025,1.011820,B
8,2.066358,0.997084,1.392861,B
9,1.855967,1.492122,0.879646,B


In [206]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

plotly.tools.set_credentials_file(username='marius92', api_key='4naxu3XiGZTGniWO02z1')

In [317]:
data = []
color = ['#0D76BF', '#43B02A', '#F93822', '#3E332E', '#FFD700', '#D62598']
list_parties= df.party.unique()

for i in range(len(list_parties)):
    
    party_now = list_parties[i]
    color_now = color[i]
    
    trace = go.Scatter(
        x= df[df['party'].str.match(party_now)]['x'], 
        y= df[df['party'].str.match(party_now)]['y'],
        mode='markers',
        name = party_now,
        marker = dict(
                size = 24,
                color = color_now),
        hoverinfo = 'text',
        hovertext = party_now, # Change to forecasted politician/or just number of seat for party ?
    )
    data.append(trace)
    
#filename = 'Sitzverteilung im Rat von" + ' ' + city_now -> Activate in Loop across cities

summe = len(df['party'])    
_sum = str(summe)
x_annot = (df['x'].iloc[-1] / 2) + df['x'].iloc[0] # x center alignment for text
y_max = df['y'].max()

layout = go.Layout(
    legend = dict(orientation='v',
                  font = dict(family = "Old Standard TT",
                              size = 24,
                              color = 'black',
                        ),
                  x = df['x'].iloc[0] - 0.5,
                  y = df['y'].iloc[0] , 
                ),
    xaxis = dict(showgrid=False, zeroline=False, visible = False),
    yaxis = dict(showgrid=False, zeroline=False, visible = False),
    title = dict(text = '<b>Sitzverteilung im Rat von </b>' '<br> <b> Z </b>',
                 font = dict(family = 'Old Standard TT',
                             size = 32,
                             color = 'black',
                        ),
                 x = 0.575,
                 y = 0.95,
            ),
    annotations = [
        go.layout.Annotation(
            x = x_annot, # middle element of x column
            y = df['y'][0], # first element of y column
            text = _sum + ' Sitze', # Summe aller Sitze
            font = dict(
                size = 24,
                family = 'Old Standard TT',
                color = 'black',
            ),
            showarrow = False,
        )
    ]
)


fig = dict(data=data, layout = layout)

py.iplot(fig, filename = 'test')